In [1]:
import pandas as pd
import duckdb
import duckdb
import os
import glob

In [5]:
def carregar_csv_pandas_para_duckdb(pasta_input: str,
                                    caminho_banco: str,
                                    delimitador: str = ";",
                                    substituir_existente: bool = False):
    """
    Versão simples: lê CSVs usando pandas (ISO-8859-1) e cria tabelas no schema 'raw'.
    O nome da tabela é baseado no nome do arquivo (sem extensão).
    
    Args:
        pasta_input: Caminho da pasta com os arquivos CSV
        caminho_banco: Caminho do banco DuckDB
        delimitador: Delimitador do CSV (padrão: ";")
        substituir_existente: Se True, substitui tabelas existentes. Se False, pula tabelas existentes.
    """

    # Conecta ao banco
    con = duckdb.connect(caminho_banco)

    # Garante schema raw
    con.execute("CREATE SCHEMA IF NOT EXISTS raw;")

    # Encontrar todos os CSV (case-insensitive)
    arquivos = glob.glob(os.path.join(pasta_input, "*.csv")) + \
               glob.glob(os.path.join(pasta_input, "*.CSV"))

    if not arquivos:
        print("⚠️ Nenhum CSV encontrado na pasta.")
        con.close()
        return

    for caminho in arquivos:
        nome_arquivo = os.path.basename(caminho)
        # Mantém o nome original (maiúsculas) para compatibilidade com o notebook 02
        nome_tabela = os.path.splitext(nome_arquivo)[0]
        tabela_schema = f"raw.{nome_tabela}"

        print(f"📄 Lendo CSV (ISO-8859-1): {nome_arquivo}")

        # Verifica se a tabela já existe
        existe = con.execute(
            f"SELECT COUNT(*) FROM information_schema.tables "
            f"WHERE table_schema = 'raw' AND table_name = '{nome_tabela}'"
        ).fetchone()[0]

        if existe and not substituir_existente:
            print(f"⏭️  Tabela '{tabela_schema}' já existe. Pulando...")
            continue

        # Lendo CSV com pandas
        try:
            df = pd.read_csv(
                caminho,
                sep=delimitador,
                encoding="ISO-8859-1",
                dtype=str   # garante leitura estável
            )
        except Exception as e:
            print(f"❌ Erro ao ler arquivo '{nome_arquivo}': {e}")
            continue

        # Registrar o DataFrame no DuckDB
        con.register("tmp_df", df)

        # Criar tabela definitiva (substitui se existir)
        try:
            if existe and substituir_existente:
                con.execute(f"DROP TABLE IF EXISTS {tabela_schema};")
                print(f"🗑️  Tabela existente removida: {tabela_schema}")
            
            con.execute(f"CREATE TABLE {tabela_schema} AS SELECT * FROM tmp_df;")
            print(f"✅ Tabela criada: {tabela_schema} ({len(df)} linhas)")
        except Exception as e:
            print(f"❌ Erro ao criar tabela '{tabela_schema}': {e}")
            continue

    con.close()
    print("✅ Carga concluída com sucesso!")

In [6]:
# Caminho do arquivo (pode ser CSV, Parquet, etc)
PATH_DIRETORIO = "data_input"

# Caminho do banco DuckDB (arquivo .db)
CAMINHO_DUCKDB = "bd/dev.duckdb"

# Cria a conexão com o banco DuckDB
con = duckdb.connect(CAMINHO_DUCKDB)

In [7]:
carregar_csv_pandas_para_duckdb(pasta_input=PATH_DIRETORIO, caminho_banco=CAMINHO_DUCKDB, delimitador=";")

📄 Lendo CSV (ISO-8859-1): MICRODADOS_CADASTRO_CURSOS_2017.CSV
⏭️  Tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2017' já existe. Pulando...
📄 Lendo CSV (ISO-8859-1): MICRODADOS_CADASTRO_CURSOS_2018.CSV
⏭️  Tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2018' já existe. Pulando...
📄 Lendo CSV (ISO-8859-1): MICRODADOS_CADASTRO_CURSOS_2023.CSV
⏭️  Tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2023' já existe. Pulando...
📄 Lendo CSV (ISO-8859-1): MICRODADOS_CADASTRO_CURSOS_2024.CSV
⏭️  Tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2024' já existe. Pulando...
📄 Lendo CSV (ISO-8859-1): MICRODADOS_CADASTRO_CURSOS_2017.CSV
⏭️  Tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2017' já existe. Pulando...
📄 Lendo CSV (ISO-8859-1): MICRODADOS_CADASTRO_CURSOS_2018.CSV
⏭️  Tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2018' já existe. Pulando...
📄 Lendo CSV (ISO-8859-1): MICRODADOS_CADASTRO_CURSOS_2023.CSV
⏭️  Tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2023' já existe. Pulando...
📄 Lendo CSV (ISO-8859-1): MICRODADOS_CADASTRO_CURSOS_2024.CSV
⏭️  Tab

In [8]:
con.close()